In [52]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, TfidfTransformer
from sklearn.datasets import load_files
import pandas as pd
import numpy as np
from function import str_to_list
import re

In [53]:
from wordcloud import WordCloud
from pythainlp.corpus import thai_stopwords
th_stop_origin = tuple(thai_stopwords())

In [100]:
df = pd.read_csv('csv/new_data.csv')

Data Preparation

In [101]:
def text_process(text):
    text = re.sub("\[|\]|'|,"," ",text).replace(" ", "")
    return text
def text_process_save_comma(text): ##save ,
    text = re.sub("\[|\]|'|"," ",text).replace(" ", "")
    text = re.sub(r'[0-9]+'," ",text)
    return text
    
# df['text_replace'] = df['text'].apply(text_process)
df['text_replace'] = df['text'].apply(text_process_save_comma)
df

,post_type,text,text_replace
0,2,"['หา', 'สัญญา', 'หอ', 'มจพ', 'ย้าย', 'ปลาย', '...","หา,สัญญา,หอ,มจพ,ย้าย,ปลาย,พฤศ,จิ,ธันวา,เรท,ราค..."
1,1,"['ขาย', 'ตั๋วหนัง', 'black', 'panther', 'โรง',...","ขาย,ตั๋วหนัง,black,panther,โรง,เมเจอร์,gateway..."
2,1,"['อนุญาต', 'ขาย', 'ขาย', 'ตู้', 'เย็น', 'Midea...","อนุญาต,ขาย,ขาย,ตู้,เย็น,Midea, ,q, ,บาท,ขาย,เค..."
3,1,"['XX', 'คน', 'เช่า', 'XX', 'อนุญาต', 'ประกาศ',...","XX,คน,เช่า,XX,อนุญาต,ประกาศ,ประชาสัมพันธ์,สำหร..."
4,1,"['ขาย', 'สัญญา', 'หอ', 'หมู่บ้าน', 'สิ', 'ริน'...","ขาย,สัญญา,หอ,หมู่บ้าน,สิ,ริน, ,ห้อง,ชั้น, , ,ห..."
...,...,...,...
1403,1,"['ขาย', 'สัญญา', 'S9', 'Residence', 'ชั้น', '9...","ขาย,สัญญา,S ,Residence,ชั้น, ,ห้อง, ,ต้น,เดือน..."
1404,2,"['แนว', 'ข้อ', 'สอบ', 'รอบ', 'สอบตรง', 'สำหรับ...","แนว,ข้อ,สอบ,รอบ,สอบตรง,สำหรับ,ปวส,รู้,ข้อ,สอบ,..."
1405,0,"['คน', 'ลืม', 'ศาลยีราฟ', 'ฝาก', 'ร้าน', 'ถ่าย...","คน,ลืม,ศาลยีราฟ,ฝาก,ร้าน,ถ่าย,เอกสาร,ริม,น้ำ"
1406,2,"['พี่', 'คน', 'รอบ', '2', 'สอบ', 'ข้อ', 'แนะนำ...","พี่,คน,รอบ, ,สอบ,ข้อ,แนะนำ,สอบ,คณิต,อังกฤษ,Bms..."


In [5]:
# import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS
# df_pos = df[df['post_type'] == 0]
# pos_word_all = " ".join(text for text in df_pos['text'])
# reg = r"[ก-๙a-zA-Z']+"
# fp = 'content\Ethan.ttf'
# wordcloud = WordCloud(stopwords=th_stop_origin, background_color = 'white', max_words=2000, height = 2000, width=4000, font_path=fp, regexp=reg).generate(pos_word_all)
# plt.figure(figsize = (16,8))
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.show()

In [6]:
# df_pos = df[df['post_type'] == 1]
# pos_word_all = " ".join(text for text in df_pos['text'])
# reg = r"[ก-๙a-zA-Z']+"
# fp = 'content\Ethan.ttf'
# wordcloud = WordCloud(stopwords=th_stop_origin, background_color = 'white', max_words=2000, height = 2000, width=4000, font_path=fp, regexp=reg).generate(pos_word_all)
# plt.figure(figsize = (16,8))
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.show()

In [7]:
# df_pos = df[df['post_type'] == 3]
# pos_word_all = " ".join(text for text in df_pos['text'])
# reg = r"[ก-๙a-zA-Z']+"
# fp = 'content\Ethan.ttf'
# wordcloud = WordCloud(stopwords=th_stop_origin, background_color = 'white', max_words=2000, height = 2000, width=4000, font_path=fp, regexp=reg).generate(pos_word_all)
# plt.figure(figsize = (16,8))
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.show()

In [102]:
X = df['text_replace']
# X = df['text']
y = df["post_type"]

In [103]:
X[0]

'หา,สัญญา,หอ,มจพ,ย้าย,ปลาย,พฤศ,จิ,ธันวา,เรท,ราคา,ห้อง, ,บาท'

In [104]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25, random_state=0)

**Test count vector**

In [105]:
from sklearn.feature_extraction.text import CountVectorizer

cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(X_train)

In [106]:
cvec.vocabulary_

{'หา': 2573,
 'สัญญา': 2355,
 'หอ': 2515,
 'S ': 288,
 'รีบ': 2066,
 'เดือน': 2858,
 'ขาย': 854,
 'หอพัก': 2529,
 'U': 348,
 'Life': 194,
 'Apartment': 26,
 'วัด': 2206,
 'ตึก': 1300,
 ' ': 1,
 'ชั้น': 1087,
 'สนใจ': 2296,
 'คุย': 989,
 'ราคา': 2046,
 'กันได้': 768,
 'เม': 2952,
 'ย': 1958,
 'ห้อง': 2608,
 'สุดท้าย': 2423,
 'ไม่ต้อง': 3344,
 'จ่าย': 1064,
 'เตียง': 2870,
 'ฟุต': 1812,
 'แอร์': 3195,
 'ตัว': 1258,
 'โต๊ะทำงาน': 3229,
 'โต๊ะ': 3226,
 'ยาวติดกัน': 1976,
 'ชั้นวางของ': 1088,
 'ตู้เสื้อผ้า': 1316,
 'คีย์': 977,
 'การ์ด': 781,
 'ใบ': 3310,
 'ระเบียง': 2029,
 'ไฟ': 3335,
 'เส้น': 3032,
 'led': 524,
 'รอบ': 2022,
 'ติดต่อ': 1287,
 'ขอดู': 838,
 'สอบถามรายละเอียด': 2338,
 'inbox': 501,
 'Linekwaneii': 196,
 'Tell': 337,
 'หิ้ว': 2597,
 'น้า': 1491,
 'เด้ออ': 2861,
 'มิถุน้า': 1924,
 'ทัก': 1363,
 'โลยยย': 3279,
 'พัก': 1735,
 'mentor': 549,
 'มิถุนายน': 1923,
 'จอด': 1030,
 'มอเตอร์ไซค์': 1884,
 'รถ': 2000,
 'ยนต์': 1962,
 'ริม': 2062,
 'คลอง': 912,
 'แบ่ง': 3127,
 'หัวปากกา': 

In [107]:
c_feat[:,:20].todense()

matrix([[ 0,  0,  0, ...,  0,  0,  0],
        [ 0, 11,  0, ...,  0,  0,  0],
        [ 0,  1,  0, ...,  0,  0,  0],
        ...,
        [ 0,  2,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  1,  0, ...,  0,  0,  0]], dtype=int64)

In [108]:
from sklearn.naive_bayes import MultinomialNB
cls = MultinomialNB(alpha=1.0, fit_prior=True)
# transform the list of text to tf-idf before passing it to the model
cls.fit(cvec.transform(X_train), y_train)
 
from sklearn.metrics import classification_report, accuracy_score
 
y_pred = cls.predict(cvec.transform(X_test))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8863636363636364
              precision    recall  f1-score   support

           0       0.90      0.95      0.92        57
           1       0.85      0.90      0.88       125
           2       0.91      0.85      0.88       170

    accuracy                           0.89       352
   macro avg       0.89      0.90      0.89       352
weighted avg       0.89      0.89      0.89       352



In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
  ngram_range=(2, 3),              # อยากวิเคราะห์แบบ 2 และ 3 คำ ติดกัน
  min_df=5,                        # ขั้นต่ำของ Doc Freq ของ Term
  max_features=30,               # ใช้แค่ 3,000 Term แรก
  decode_error='ignore'
  )
X_train_vectorized = vectorizer.fit_transform(X_train)

In [47]:
first_vector_tfidfvectorizer=X_train_vectorized[0] 
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=vectorizer.get_feature_names_out(), columns=["tfidf"])

In [48]:
print(type(list(vectorizer.vocabulary_)))
data = list(vectorizer.vocabulary_)
for i in data:
    print(i)
print(type(vectorizer.idf_))

<class 'list'>
ขายส ญญาหอพ
อนส ดท
เส อผ
องทำน ำอ
องซ กผ
วงหน า1เด
ดต อร
มพ นธ
านหน งส
งส อเก
อเก าอ
าอ 2ต
องเด อนส
งส อเก าอ
องเด อนส ดท
กผ าช
เคร องทำน
นต เย
องซ กผ าช
เคร องทำน ำอ
หาบ านเช
กผ าหยอดเหร
ทยาล ยเทคโนโลย
พระจอมเกล าพระนครเหน
ะเก าอ
อผ าโต
เส อผ าโต
เย นเคร
ขายต เย
ดต อเบอร
<class 'numpy.ndarray'>


Built model

In [27]:
from sklearn.naive_bayes import MultinomialNB
cls = MultinomialNB(alpha=1.0, fit_prior=True)
# transform the list of text to tf-idf before passing it to the model
cls.fit(vectorizer.fit_transform(X_train), y_train)
 
from sklearn.metrics import classification_report, accuracy_score
 
y_pred = cls.predict(vectorizer.fit_transform(X_test))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.5568181818181818
              precision    recall  f1-score   support

           0       0.44      0.25      0.31        57
           1       0.54      0.33      0.41       125
           3       0.58      0.83      0.68       170

    accuracy                           0.56       352
   macro avg       0.52      0.47      0.47       352
weighted avg       0.54      0.56      0.52       352



In [15]:
import pickle

filename = "./model/check_type.sav"
filenamevec = "./model/vectorizer.sav"

#save model
pickle.dump(cls,open(filename,"wb"))
pickle.dump(vectorizer,open(filenamevec,"wb"))
